# Aula 3 - Parte Prática - Redução de Variância e Função Valor

## Introdução

Nesse terceiro notebook vamos realizar experimentos com algumas técnicas estatísticas para redução de variância do estimador do *policy gradients* do algoritmo REINFORCE visto na última aula.


$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^{T-1} \nabla_\theta \log \pi_\theta(\mathbf{a}_t|\mathbf{s}_t) \left ( \left( \sum_{k=t}^{T-1} r_{k} \right)  - b(\mathbf{s}_{t}) \right ) \right]
$$


### Objetivos:

- Relacionar as propriedades do estimador REINFORCE com a performance do agente
- Verificar experimentalmente o efeito de redução de variância do estimador de Policy Gradient calculado com *reward-to-go*
- Incorporar a função Valor como *baseline* para os retornos das trajetórias no REINFORCE
- Familiarizar-se com o aprendizado de função Valor via regressão sobre os retornos das trajetórias 

### Imports

> **Atenção:** não se esqueça de executar todos os `imports` necessários antes prosseguir com o tutorial.

In [ ]:
import logging

import gym
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

from utils.agent import RLAgent
from utils.memory import OnPolicyReplay
from utils.networks import build_discrete_policy, build_value_network, get_optimizer
import utils.runner
from utils.viz import *


tf.get_logger().setLevel("ERROR")     # ignore TensorFlow warnings
gym.logger.set_level(logging.ERROR)   # ignore OpenAI Gym warnings

## 0. Configurações 

> **Atenção**: a fim de tornar o problema do `CartPole` um pouco mais desafiador, vamos utilizar nesse notebook a versão `v1` que aumenta o tamanho máximo de uma trajetória (i.e., `env.spec.max_episode_steps`) e também o retorno mínimo para resolver a tarefa (e.g, `env.spec.reward_threshold`).

In [ ]:
env = gym.make("CartPole-v1")
print(env.spec.max_episode_steps, env.spec.reward_threshold)

In [ ]:
config = {
    # policy network
    "hidden_layers": [64, 64],
    "activation": "relu",

    # optimization
    "optimizer": "adam",
    "learning_rate": 1e-3,

    # training
    "train_batch_size": 3000,
}

## 1. REINFORCE

Na classe REINFORCE abaixo re-implementamos o policy gradient que estudamos na última aula e adicionamos um argumento adicional ao construtor `__init__` da classe correspondente a função `postprocessing` que processa um `batch` de recompensas. Utilizaremos esse argumento na próxima seção para passar uma função que computará o *reward-to-go* dos passos de uma trajetória.

In [ ]:
class REINFORCE(RLAgent):
    """
    Classe que implementa os componentes de um agente que aprende pelo REINFORCE.

    Args:
        obs_space:       especificação do espaço de observações do ambiente.
        action_space:    especificação do espaço de ações do ambiente.
        config (dict):   configurações de hiper-parâmetros.
        postprocessing:  função que processa um batch de recompensas.
    """
    
    def __init__(self, obs_space, action_space, config, postprocessing=None):
        super(REINFORCE, self).__init__(obs_space, action_space, config)
        
        self.memory = OnPolicyReplay()
        self.policy = build_discrete_policy(self.obs_space, self.action_space, config["hidden_layers"], config["activation"])
        self.optimizer = get_optimizer(config["optimizer"], config["learning_rate"])

        self.postprocessing = postprocessing or np.sum

    def act(self, obs):
        """
        Escolhe uma ação para ser tomada dada uma observação do ambiente.
        
        Args: 
            obs: observação do ambiente.
        
        Return:
            action: ação válida dentro do espaço de ações.
        """
        return self._act(obs).numpy()
    
    @tf.function
    def _act(self, obs):
        action_dist = self.policy(obs[None,:])
        return action_dist.sample()[0]

    def observe(self, obs, action, reward, next_obs, done):
        """
        Registra na memória do agente uma transição do ambiente.

        Args:
            obs:            observação do ambiente antes da execução da ação.
            action:         ação escolhida pelo agente.
            reward (float): escalar indicando a recompensa obtida após a execução da ação.
            next_obs:       nova observação recebida do ambiente após a execução da ação.
            done (bool):    True se a nova observação corresponde a um estado terminal, False caso contrário.

        Return:
            None
        """
        self.memory.update(obs, action, reward, next_obs, done)

    def learn(self):
        """
        Método de treinamento do agente. A partir das experiências de sua memória,
        o agente aprende um novo comportamento.

        Args: 
            None

        Return:
            None
        """
        if self.memory.batch_size < self.config["train_batch_size"]:
            return
        
        batch = self.memory.sample()

        with tf.GradientTape() as tape:
            loss = self._loss_pg_fn(batch)
            gradients = tape.gradient(loss, self.policy.trainable_weights)

        self.optimizer.apply_gradients(zip(gradients, self.policy.trainable_weights))
      
        return loss

    def _loss_pg_fn(self, batch):
        """
        Calcula a função loss do policy gradients para um `batch` de trajetórias/episódios.
        
        Um `batch` agrega listas de arrays n-dimensionais. Cada lista (e.g., batch["states"],
        batch["actions"], batch["rewards"]) tem o tamanho do número de episódios. Por exemplo,
        batch["states"][k] devolve um array n-dimensional para o k-ésimo episódio. Este array
        tem como primeira dimensão o número de timesteps do k-ésimo episódio.

        Args:
            batch (Dict[str, List[np.ndarray]]): dicionário para acesso às listas de estados, ações e recompensas. 
        
        Return:
            loss (tf.Tensor): média sobre os episódios do surrogate loss function.
            
        """
        states, actions, rewards = batch["states"], batch["actions"], batch["rewards"]
        
        n_episodes = len(rewards)

        loss = 0.0
        for episode in range(n_episodes):
            action_dist = self.policy(states[episode])
            log_prob = action_dist.log_prob(actions[episode])
            R_t = self.postprocessing(rewards[episode])
            loss += - tf.reduce_sum(log_prob * R_t)

        loss /= n_episodes
            
        return loss

Execute o código abaixo rodar o mesmo experimento do **REINFORCE** um número dado de vezes (i.e., `n_trials`):

In [ ]:
n_trials = 5
total_timesteps = 1_000_000
agent_cls = REINFORCE
postprocessing = None

timesteps, total_rewards, avg_total_rewards = utils.runner.run_experiments(
    n_trials, env, agent_cls, config, postprocessing, total_timesteps)

plot_experiments(env, timesteps, total_rewards, avg_total_rewards)

Execute o código abaixo para visualizar a política aprendida pelo algoritmo REINFORCE: 

In [ ]:
n_episodes = 10
_ = evaluate(agent, env, n_episodes, render=True)

## 2. Policy Gradients: ignorando recompensas passadas

Inicialmente vamos substituir o retorno (i.e., recompensa total acumulada) de um episódio pelo *reward-to-go*:
$$
R_t = \sum_{k=t}^{T-1} r_k
$$

Lembre-se que na fórmula original do *policy gradient* o retorno da trajetória inteira é utilizado como sinal de reforço para todos os passos $t$ da mesma forma. Em outras palavras, o *score function* $\nabla_\theta \log \pi_\theta(\mathbf{a}_t|\mathbf{s}_t)$ do passo $t$ é ponderado com recompensas tanto do passado ($t' < t$) quanto do futuro ($t' \geq t$)! Isso apenas adiciona ruído no estimador; impactando de forma negativa a sua variância. É importante notar que a escolha de uma ação só tem influência no retorno futuro, isto é, a partir do momento da tomada da ação.

Ignorando recompensas passadas no REINFORCE podemos "filtrar" esse ruído de maneira que o estimador do policy gradient derivado anteriormente terá sua variância reduzida (embora seu valor esperado não se altere):

$$
\begin{align*}
\nabla_\theta J(\theta) &= \mathbb{E}_{\tau \sim \pi_\theta} \left [ \sum_{t=0}^T \nabla_\theta \log \pi_\theta(\mathbf{a}_t|\mathbf{s}_t) \left( \sum_{t=1}^{T-1} r_t \right) \right ] \\
&= \mathbb{E}_{\tau \sim \pi_\theta} \left [ \sum_{t=0}^T \nabla_\theta \log \pi_\theta(\mathbf{a}_t|\mathbf{s}_t) \left( \sum_{k=t}^{T-1} r_k \right) \right ]
\end{align*}
$$

> **Atenção**: note como a mudança do subescrito $k$ faz diferença na última igualdade! Na primeira igualdade (i.e.,

### 2.1 Reward-to-Go 

No código abaixo implementamos o *reward-to-go* para uma trajetória. Note que a função do `np.cumsum` do NumPy calcula a soma acumulada a cada passo do array n-dimensional de `rewards`. Para maiores detalhes consulte a documentação de [numpy.cumsum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cumsum.html).

In [ ]:
def compute_reward_to_go(rewards):
#     returns = []
#     total_reward = 0.0

#     for i in range(len(rewards) - 1, -1, -1):
#         reward = rewards[i]
#         total_reward += reward
#         returns.append(total_reward)

#     returns = returns[::-1]

#     return np.array(returns)
    return np.cumsum(rewards[::-1])[::-1] # essa linha é equivalente ao for-loop comentado acima, porém mais eficiente!


Execute o código abaixo rodar o mesmo experimento do **REINFORCE+reward-to-go** um número dado de vezes (i.e., `n_trials`):

In [ ]:
n_trials = 5
total_timesteps = 1_000_000
agent_cls = REINFORCE
postprocessing = compute_reward_to_go

timesteps, total_rewards, avg_total_rewards = utils.runner.run_experiments(
    n_trials, env, agent_cls, config, postprocessing, total_timesteps)

plot_experiments(env, timesteps, total_rewards, avg_total_rewards)

Execute o código abaixo para visualizar a política aprendida pelo algoritmo **REINFORCE + reward-to-go**: 

In [ ]:
n_episodes = 10
_ = evaluate(agent, env, n_episodes, render=True)

## 3. Policy Gradients: adicionando referências para os retornos (baseline)

Na classe `VPG` (i.e., *Vanilla Policy Gradients*) especializamos a classe do algoritmo REINFORCE para incorporar uma referência para os retornos dos episódios (i.e., *baseline*). Neste notebook utilizaremos a função Valor como função *baseline*.


> **Observação**: embora não haja consenso na literatura, note que estamos usando o nome de *Vanilla Policy Gradients* para denotar o algoritmo REINFORCE com *baseline* calculado pela função Valor.


Lembre-se que a a **função Valor** tenta estimar o retorno médio a partir de um dado estado $\mathbf{s}$:
$$
V_\phi(\mathbf{s}) \approx V^{\pi_\theta}(\mathbf{s}) = \mathbb{E}_{\tau \sim \pi_\theta} \left [ \sum_{t=1}^{T-1} r_t~\middle |~\mathbf{s}_0 = \mathbf{s} \right ]
$$

Para aprendermos os parâmetros $\phi$ do aproximador da função Valor tentaremos resolver um problema de [regressão](https://en.wikipedia.org/wiki/Nonlinear_regression) usando MSE (i.e., *[Mean Squared Error](https://en.wikipedia.org/wiki/Mean_squared_error)*) em cada época de treinamento (e.g., cada vez que o método `learn` é chamado dentro do ciclo de interação agente-ambiente):

$$
\phi_k = \arg\min_{\phi} \mathbb{E}_{\mathbf{s} \sim d^{\pi_{\theta}}} \left [ (V_{\phi}(\mathbf{s_t}) - R_t)^2 \right ]
$$
onde $R_t = \sum_{k=t}^{T-1}$ corresponde ao *reward-to-go* a partir do passo $t$ e $d^{\pi_\theta}$ corresponde a distribuição de visitação de estados induzida pela política atual $\pi_\theta$. 


In [ ]:
class VPG(REINFORCE):
    
    def __init__(self, obs_space, action_space, config, postprocessing=None):
        super(VPG, self).__init__(obs_space, action_space, config, postprocessing)
        
        config = config["value_fn"]
        self.value_fn = build_value_network(obs_space, config["hidden_layers"], config["activation"])
        self.value_fn.compile(get_optimizer(config["optimizer"], config["learning_rate"]), loss="MSE")
    
    def learn(self):
        """
        Método de treinamento do agente. A partir das experiências de sua memória,
        o agente aprende um novo comportamento.

        Args: 
            None

        Return:
            None
        """
        if self.memory.batch_size < self.config["train_batch_size"]:
            return
        
        batch = self.memory.sample()

        value_loss = self._train_value_fn(batch)
        policy_loss = self._train_policy_fn(batch)
      
        return policy_loss, value_loss
    
    def _train_policy_fn(self, batch):
        """Executa um passo de gradiente ascedente para melhorar a política."""
        with tf.GradientTape() as tape:
            policy_loss = self._loss_pg_fn(batch)
            gradients = tape.gradient(policy_loss, self.policy.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.policy.trainable_weights))
        return policy_loss
    
    def _train_value_fn(self, batch):
        """Executa vários passos do gradiente ascedente para melhorar a função Valor (e.g., baseline)."""
        states, rewards = batch["states"], batch["rewards"]
        R_t = list(map(self.postprocessing, rewards))
        
        states = np.concatenate(states, axis=0)
        R_t = np.concatenate(R_t, axis=0)

        batch_size = len(states)
        self.value_fn.fit(states, R_t, epochs=self.config["value_fn"]["epochs"], batch_size=batch_size, verbose=0)

        value_loss = self.value_fn.evaluate(states, R_t, verbose=0)
        return value_loss
    
    def _loss_pg_fn(self, batch):
        """Calcula surrogate loss do policy gradient considerando a função de baseline."""
        states, actions, rewards = batch["states"], batch["actions"], batch["rewards"]
        n_episodes = len(states)

        loss = 0.0
        for episode in range(n_episodes):
            action_dist = self.policy(states[episode])
            log_prob = action_dist.log_prob(actions[episode])

            R_t = self.postprocessing(rewards[episode])
            baseline = self.value_fn(states[episode])

            loss += - tf.reduce_sum(log_prob * (R_t - baseline))

        loss /= n_episodes
            
        return loss


Execute o código abaixo rodar o mesmo experimento do **Vanilla Policy Gradients** um número dado de vezes (i.e., `n_trials`):

In [ ]:
n_trials = 5
total_timesteps = 1_000_000

config = {
    **config,
    "value_fn": {
        # network
        "hidden_layers": [64, 64],
        "activation": "relu",
        
        # optimization
        "optimizer": "rmsprop",
        "learning_rate": 3e-3,
        "epochs": 10,
    }
}

agent_cls = VPG
postprocessing = compute_reward_to_go

timesteps, total_rewards, avg_total_rewards = utils.runner.run_experiments(
    n_trials, env, agent_cls, config, postprocessing, total_timesteps)

plot_experiments(env, timesteps, total_rewards, avg_total_rewards)

Execute o código abaixo para visualizar a política aprendida pelo algoritmo **Vanilla Policy Gradient**: 

In [ ]:
n_episodes = 10
_ = evaluate(agent, env, n_episodes, render=True)